In [11]:
import copy

In [6]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

In [7]:
def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

# CodeSearchNet dataset

In [14]:
def addClass(method):
    return "public class Test {\n"+method+"\n}"

# Encoding

In [225]:
parse_tree = getJsonData("F:\\GDrive RA\\RA\\Code Translation\\mavenproject1\\parse_tree.json")

In [226]:
parse_tree = parse_tree['compilationUnit'][0]['typeDeclaration'][1]['classDeclaration'][2]['classBody'][1]

In [541]:
parse_tree

{'classBodyDeclaration': [{'modifier': [{'classOrInterfaceModifier': [{'type': 33,
       'text': 'private'}]}]},
  {'memberDeclaration': [{'methodDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
           'text': 'NumberFormat'}]}]},
      {'type': 100, 'text': 'parseFormatter'},
      {'formalParameters': [{'type': 57, 'text': '('},
        {'formalParameterList': [{'formalParameter': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                 'text': 'Context'}]}]},
            {'variableDeclaratorId': [{'type': 100, 'text': 'context'}]}]},
          {'type': 64, 'text': ','},
          {'formalParameter': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                 'text': 'Arguments'}]}]},
            {'variableDeclaratorId': [{'type': 100, 'text': 'args'}]}]}]},
        {'type': 58, 'text': ')'}]},
      {'methodBody': [{'block': [{'type': 59, 'text': '{'},
          {'blockStatement': [{'localVariableDeclarationStatement': [{'localVari

# Visualize Tree

In [15]:
def travarse(tree):
    if 'type' in tree:
        print(tree['text'], end=" ")
        if tree['text'] in [';', '{', '}']:
            print()
        return
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            travarse(child)

In [16]:
def tree2code(tree, indentation):
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        return code, indentation
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        code = ""
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
        return code, indentation

In [17]:
def printCode(tree):
    code, _ = tree2code(tree, 0)
    print(code)
    print("Count:", len(code.split()))

# Compression

In [18]:
import ast
def get_compressed_func():
    global compression_number
    compression_number += 1
    return "$F"+str(compression_number)

In [19]:
abstract_code_dict = {}
compression_number = 0

In [20]:
def build_n_compress(tree, build_dict):
    var_list = []
    global abstract_code_dict
    global num_to_compress
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, var_list)
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [])
        
        code = ""
        leaf_count = 0
        id_count = 0
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl) = build_n_compress(child, build_dict)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]
                
            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                # Build the dictionary or compress the tree
                if build_dict:
                    if abstract_code not in abstract_code_dict:
                        compressed_func = get_compressed_func()
                        abstract_code_dict[abstract_code] = {
                            'count': 0, 
                            'compressed_name': compressed_func,
                            'num_params': len(var_list)
                        }
                    abstract_code_dict[abstract_code]['count'] += 1
                else:
                    # If the abstracted code is within the range, replace
                    # the children with a single leaf node containing whole code
                    # If the list is empty, only one token.
                    if abstract_code in abstract_code_keys[:num_to_compress]:
                        compressed_func = abstract_code_dict[abstract_code]['compressed_name']
                        compressed_func += ' ( ' +' , '.join(var_list)+' )' if var_list else ''
                        tree[parent] = [{"type": "compressed", "text": compressed_func}]
                        
            #print(parent, ":\t",code, leaf_count, id_count, var_list)
            return (code, leaf_count, id_count, var_list)
        
        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)

        (code, leaf_count, i, vl) = build_n_compress(children[0], build_dict)
        id_count += i
        var_list += [v for v in vl if v not in var_list]
        
        #print(parent, ":\t",code, leaf_count, id_count, var_list)
        return (code, leaf_count, id_count, var_list)

In [109]:
num_to_compress = 30000

In [132]:
i = 1011
parse_tree1 = copy.deepcopy(all_data[i])
(code, leaf_count, id_count, var_list) = build_n_compress(parse_tree1, False)

In [133]:
printCode(all_data[i])

public List < List < String > > getFilteredNamedPolicy ( String ptype , int fieldIndex , String ... fieldValues ) { 
	return model . getFilteredPolicy ( "p" , ptype , fieldIndex , fieldValues ) ; 
	} 

Count: 36


In [134]:
printCode(parse_tree1)

public $F17095 getFilteredNamedPolicy $F17986 ( ptype , fieldIndex , fieldValues ) { 
	return model . getFilteredPolicy ( $F15602 ( ptype , fieldIndex , fieldValues ) ) ; 
	} 

Count: 28


In [56]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)
abstract_code_keys[:50]

['( )',
 '( $id0 == null )',
 '@ Override',
 '( STRING00 )',
 '( $id0 != null )',
 'IllegalArgumentException ( STRING00 )',
 'new IllegalArgumentException ( STRING00 )',
 'throw new IllegalArgumentException ( STRING00 ) ;',
 'break ;',
 '{ throw new IllegalArgumentException ( STRING00 ) ; }',
 '$id0 ( $id1 , $id2 , $id3 )',
 '$id0 . body ( )',
 '$id0 . length ( )',
 'return $id0 . body ( ) ;',
 '{ return $id0 . body ( ) ; }',
 '$id0 . get ( $id1 )',
 '- INT00',
 'if ( $id0 == null ) { throw new IllegalArgumentException ( STRING00 ) ; }',
 'return null ;',
 '$id0 . size ( )',
 '< String >',
 'return false ;',
 '$id0 ( $id1 , $id2 , $id3 , $id4 )',
 'int $id0 = INT00 ;',
 '< Void >',
 '$id0 . toString ( )',
 'return ;',
 '< >',
 '$id0 . add ( $id1 )',
 '$id0 . add ( $id1 ) ;',
 '< ResponseBody >',
 'Response < ResponseBody >',
 '$id0 . append ( STRING00 )',
 'byte [ ]',
 '{ return $id0 ; }',
 '( String $id0 , String $id1 )',
 'STRING00 + $id0 + STRING00',
 '< ? >',
 'return true ;',
 '< 

In [63]:
for x in abstract_code_keys[:101]:
    print(x, "-->", abstract_code_dict[x])

( ) --> {'count': 13831, 'compressed_name': '$F1', 'num_params': 0}
( $id0 == null ) --> {'count': 6462, 'compressed_name': '$F4', 'num_params': 1}
@ Override --> {'count': 5896, 'compressed_name': '$F216', 'num_params': 0}
( STRING00 ) --> {'count': 5379, 'compressed_name': '$F110', 'num_params': 0}
( $id0 != null ) --> {'count': 4520, 'compressed_name': '$F529', 'num_params': 1}
IllegalArgumentException ( STRING00 ) --> {'count': 3727, 'compressed_name': '$F2762', 'num_params': 0}
new IllegalArgumentException ( STRING00 ) --> {'count': 3727, 'compressed_name': '$F2763', 'num_params': 0}
throw new IllegalArgumentException ( STRING00 ) ; --> {'count': 3726, 'compressed_name': '$F2764', 'num_params': 0}
break ; --> {'count': 3654, 'compressed_name': '$F1446', 'num_params': 0}
{ throw new IllegalArgumentException ( STRING00 ) ; } --> {'count': 3567, 'compressed_name': '$F2765', 'num_params': 0}
$id0 ( $id1 , $id2 , $id3 ) --> {'count': 3091, 'compressed_name': '$F3614', 'num_params': 4}


In [66]:
abstract_code_dict[abstract_code_keys[15000]]

{'count': 6, 'compressed_name': '$F194970', 'num_params': 0}

In [52]:
dump_dict(abstract_code_dict, 'abstract_code_dict.json')

In [7]:
abstract_code_dict = getJsonData('../CodeSearchNet/abstract_code_dict.json')

In [8]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)

## Batch data

In [14]:
all_data = getJsonData("../CodeSearchNet/java_train_3.json")

In [15]:
all_data[0]

{'classBodyDeclaration': [{'modifier': [{'classOrInterfaceModifier': [{'type': 35,
       'text': 'public'}]}]},
  {'memberDeclaration': [{'methodDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
           'text': 'OmemoDevice'}]}]},
      {'type': 100, 'text': 'getOwnDevice'},
      {'formalParameters': [{'type': 57, 'text': '('},
        {'type': 58, 'text': ')'}]},
      {'methodBody': [{'block': [{'type': 59, 'text': '{'},
          {'blockStatement': [{'statement': [{'type': 42,
               'text': 'synchronized'},
              {'parExpression': [{'type': 57, 'text': '('},
                {'expression': [{'primary': [{'type': 100, 'text': 'LOCK'}]}]},
                {'type': 58, 'text': ')'}]},
              {'block': [{'type': 59, 'text': '{'},
                {'blockStatement': [{'localVariableDeclarationStatement': [{'localVariableDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                           'text': 'BareJid'}]}]},
    

In [8]:
abstract_code_dict = {}
compression_number = 0

In [ ]:
for i in range(len(all_data)):
    if i%1000 == 0: print("[", i, "]") 
    data1 = copy.deepcopy(all_data[i])
    (code, leaf_count, id_count, var_list) = build_n_compress(data1, True)
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)

[ 0 ]
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  1.0f
Literal Ev

[ 9000 ]
Literal Evaluation Exception:  60L
Literal Evaluation Exception:  60L
Literal Evaluation Exception:  1000L
Literal Evaluation Exception:  1D
Literal Evaluation Exception:  0D
Literal Evaluation Exception:  0L
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  255.0f
Literal Evaluation Exception:  0.75f
Literal Evaluation Exception:  0.75f
Literal Evaluation Exception:  0.0f
Literal Evaluation Exception:  0.0f
Literal Evaluation Exception:  0.0f
Literal Evaluation Exception:  0.0f
Literal Evaluation Exception:  1f
Literal Evaluation Exception:  1f
Literal Evaluation Exception:  2f
Literal Evaluation Exception:  0.2f
Literal Evaluation Exception:  360f
Literal Evaluation Exception:  90f
Literal Evaluation Exception:  2f
Literal Evaluation Exception:  2f
Literal Evaluation Exception:  0.5f
Literal Evaluation Exception:  1.0f
Literal Evaluation Exception:  0.0f
Literal Evaluation Exception:  360f
Literal Evaluation Exception:  360f
Literal Evaluation Exception:  36

Literal Evaluation Exception:  1L
Literal Evaluation Exception:  0xffL
Literal Evaluation Exception:  0L
Literal Evaluation Exception:  0L
Literal Evaluation Exception:  0L
Literal Evaluation Exception:  1L
Literal Evaluation Exception:  0x5555555555555555L
Literal Evaluation Exception:  0x3333333333333333L
Literal Evaluation Exception:  0x3333333333333333L
Literal Evaluation Exception:  0x0f0f0f0f0f0f0f0fL
Literal Evaluation Exception:  0x5555555555555555L
Literal Evaluation Exception:  0x5555555555555555L
Literal Evaluation Exception:  0x3333333333333333L
Literal Evaluation Exception:  0x3333333333333333L
Literal Evaluation Exception:  0x0f0f0f0f0f0f0f0fL
Literal Evaluation Exception:  0x0f0f0f0f0f0f0f0fL
Literal Evaluation Exception:  0x00ff00ff00ff00ffL
Literal Evaluation Exception:  0x00ff00ff00ff00ffL
Literal Evaluation Exception:  0xffff0000L
Literal Evaluation Exception:  0xffff0000L
Literal Evaluation Exception:  0x00ff00ff00ff00ffL
Literal Evaluation Exception:  0x00ff00ff00f

## Checking Frequencies

In [13]:
abstract_code_dict

{'( )': {'count': 13831, 'compressed_name': '$F1', 'num_params': 0},
 'BareJid $id0 = $id1 ( )': {'count': 1,
  'compressed_name': '$F2',
  'num_params': 2},
 'BareJid $id0 = $id1 ( ) ;': {'count': 1,
  'compressed_name': '$F3',
  'num_params': 2},
 '( $id0 == null )': {'count': 6462,
  'compressed_name': '$F4',
  'num_params': 1},
 'return null ;': {'count': 1819, 'compressed_name': '$F5', 'num_params': 0},
 '{ return null ; }': {'count': 790,
  'compressed_name': '$F6',
  'num_params': 0},
 'if ( $id0 == null ) { return null ; }': {'count': 267,
  'compressed_name': '$F7',
  'num_params': 1},
 '( $id0 , $id1 ( ) )': {'count': 17,
  'compressed_name': '$F8',
  'num_params': 2},
 'OmemoDevice ( $id0 , $id1 ( ) )': {'count': 1,
  'compressed_name': '$F9',
  'num_params': 2},
 'new OmemoDevice ( $id0 , $id1 ( ) )': {'count': 1,
  'compressed_name': '$F10',
  'num_params': 2},
 'return new OmemoDevice ( $id0 , $id1 ( ) ) ;': {'count': 1,
  'compressed_name': '$F11',
  'num_params': 2},
 '

In [53]:
len(abstract_code_keys)

477148

In [67]:
freqs = []
for x in abstract_code_keys[:100]:
    freq = abstract_code_dict[x]['count']
    freqs.append(freq)
    print(freq, x)

13831 ( )
6462 ( $id0 == null )
5896 @ Override
5379 ( STRING00 )
4520 ( $id0 != null )
3727 IllegalArgumentException ( STRING00 )
3727 new IllegalArgumentException ( STRING00 )
3726 throw new IllegalArgumentException ( STRING00 ) ;
3654 break ;
3567 { throw new IllegalArgumentException ( STRING00 ) ; }
3091 $id0 ( $id1 , $id2 , $id3 )
2771 $id0 . body ( )
2418 $id0 . length ( )
2244 return $id0 . body ( ) ;
2244 { return $id0 . body ( ) ; }
2239 $id0 . get ( $id1 )
2222 - INT00
2110 if ( $id0 == null ) { throw new IllegalArgumentException ( STRING00 ) ; }
1819 return null ;
1813 $id0 . size ( )
1684 < String >
1524 return false ;
1482 $id0 ( $id1 , $id2 , $id3 , $id4 )
1474 int $id0 = INT00 ;
1452 < Void >
1422 $id0 . toString ( )
1411 return ;
1401 < >
1346 $id0 . add ( $id1 )
1329 $id0 . add ( $id1 ) ;
1316 < ResponseBody >
1316 Response < ResponseBody >
1291 $id0 . append ( STRING00 )
1265 byte [ ]
1227 { return $id0 ; }
1206 ( String $id0 , String $id1 )
1183 STRING00 + $id0 + STR

In [68]:
freqs[:100]

[13831,
 6462,
 5896,
 5379,
 4520,
 3727,
 3727,
 3726,
 3654,
 3567,
 3091,
 2771,
 2418,
 2244,
 2244,
 2239,
 2222,
 2110,
 1819,
 1813,
 1684,
 1524,
 1482,
 1474,
 1452,
 1422,
 1411,
 1401,
 1346,
 1329,
 1316,
 1316,
 1291,
 1265,
 1227,
 1206,
 1183,
 1136,
 1084,
 1061,
 1013,
 1011,
 1010,
 951,
 944,
 942,
 930,
 907,
 889,
 885,
 840,
 824,
 790,
 785,
 778,
 774,
 774,
 767,
 741,
 724,
 722,
 722,
 698,
 695,
 694,
 690,
 690,
 685,
 685,
 681,
 674,
 674,
 674,
 672,
 660,
 656,
 656,
 651,
 642,
 641,
 640,
 633,
 628,
 628,
 628,
 628,
 622,
 621,
 621,
 620,
 618,
 617,
 616,
 612,
 612,
 601,
 598,
 584,
 584,
 581]

In [428]:
sum(freqs)

162853

In [440]:
sum_frac = 0
frac = 0.2
sum_100 = sum(freqs)
for i in range(len(freqs)):
    sum_frac += freqs[i]
    if sum_frac >= frac*sum_100:
        break

In [441]:
i

325

In [442]:
freqs[i]

33

In [443]:
sum_frac

32574

# 1.5M data

In [ ]:
# Load 1 file at a time
# Compress it
# Remove all data with frequency 1
# Add to dict

In [10]:
all_data = getJsonData("../csn_big_only_trees/csn_big_2.json")

NameError: name 'getJsonData' is not defined

### Build Dictionary

In [21]:
abstract_code_dict = {}
compression_number = 0

In [22]:
pwd

'/home/ec2-user/SageMaker/Sub-Tree-Encoding'

In [ ]:
data_start_number = 1
data_end_number = 16


for n in range(data_start_number, data_end_number+1):
    print("=====[",n,"]=====")
    try:
        all_data = getJsonData("../csn_big_only_trees/csn_big_{}.json".format(n))
        for i in range(len(all_data)):
            if i%10000 == 0: print("[", i, "]") 
            tree_copy = copy.deepcopy(all_data[i])
            (code, leaf_count, id_count, var_list) = build_n_compress(tree_copy, True)
    except:
        print("Data csn_big_{}.json is corrupted.".format(n))
        
    for key in list(abstract_code_dict):
        if abstract_code_dict[key]['count'] < 2:
            del abstract_code_dict[key]
    # Print Time
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)

=====[ 1 ]=====
Data csn_big_1.json is corrupted.
=====[ 2 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 3 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 4 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 5 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 6 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 7 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 8 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 9 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[ 40000 ]
[ 50000 ]
[ 60000 ]
[ 70000 ]
[ 80000 ]
[ 90000 ]
=====[ 10 ]=====
[ 0 ]
[ 10000 ]
[ 20000 ]
[ 30000 ]
[

In [28]:
for key in list(abstract_code_dict):
    if abstract_code_dict[key]['count'] < 2:
        del abstract_code_dict[key]

In [29]:
dump_dict(abstract_code_dict, 'abstract_code_dict_1.5m.json')

In [ ]:
!zip abstract_code_dict_1.5m.zip abstract_code_dict_1.5m.json
!rm abstract_code_dict_1.5m.json

In [1]:
!unzip abstract_code_dict_1.5m.zip
#!rm abstract_code_dict_1.5m.zip
#abstract_code_dict = getJsonData('abstract_code_dict_1.5m.json')

Archive:  abstract_code_dict_1.5m.zip
  inflating: abstract_code_dict_1.5m.json  


In [30]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)

### Compress

In [52]:
num_to_compress = 25000
print(abstract_code_keys[num_to_compress], abstract_code_dict[abstract_code_keys[num_to_compress]])

{ $id0 . append ( STRING00 ) ; break ; } {'count': 97, 'compressed_name': '$F498473', 'num_params': 1}


In [ ]:
for n in range(data_start_number, data_end_number+1):
    print("=====[",n,"]=====")
    try:
        all_data = getJsonData("../csn_big_only_trees/csn_big_{}.json".format(n))
        compressed_codes = []
        for i in range(len(all_data)):
            if i%1000 == 0: print("[", i, "]") 
            tree_copy = copy.deepcopy(all_data[i])
            (code, leaf_count, id_count, var_list) = build_n_compress(tree_copy, False)
            code, _ = tree2code(tree_copy, 0)
            compressed_code = {"code": code}
            compressed_codes.append(compressed_code)
        dump_dict(compressed_codes, "../csn_compressed_codes/csn_compressed_{}.json".format(n))
    except Exception as e:
        print(e)
        print("Data csn_big_{}.json is corrupted.".format(n))

Data csn_big_1.json is corrupted.
[ 0 ]
[ 1000 ]
[ 2000 ]
[ 3000 ]
[ 4000 ]
[ 5000 ]
[ 6000 ]
[ 7000 ]
Data csn_big_2.json is corrupted.


Analyzing first json corruption

In [ ]:
all_data = getJsonData("../csn_big_only_trees/csn_big_1.json")

In [9]:
dump_dict({"code": "hello world"}, "../csn_compressed_codes/csn_compressed_2.json")

FileNotFoundError: [Errno 2] No such file or directory: 'csn_compressed_codes/csn_compressed_2.json'